<a href="https://colab.research.google.com/github/Woori60/Auto-star-rating-tagging-method-using-SNS-review-data/blob/master/Final_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
review1 = "Perfect Chinese Restaurant has been around for ages, even decades! It is far from Perfect but as a 24 hour business it does survive because where can you get some Chinese food in the middle of the night in Scarborough? But... for sure Perfect will not be the place for me... can I say I rather going to bed on my empty stomach?   Prices here for dim sum is dirt cheap $1.50 for all sizes All Day weekday and weekends are $1.80. The specials are from 7 AM, great if you have to work early.. I don't expect much services in cheap Chinese restaurant. I can live with servers throwing things on our tables. I can stand the plastic table cover where all my plates are sliding everywhere. I can understand that the quality of food will not be as good with the price they are charging! What I cannot stand is lukewarm to room temperature dim sum!   Shrimp Rice Roll - not only cold, the shrimps inside was like hard.  Pan Fried Turnip Cake - cold and oily.  Chicken Feet - lukewarm  Siu Mai - lukewarm  Har Gor- The only one that is tasty and hot. Now I am more than happy to pay more at other dim sum places at least I know my food is hot and steamy.. and I can even complain to the manager if they are not up to par...1.5 stars with 1 star goes to the fact they are open 24/7"
#review1 = "If you looking for a real sushi experience, go to Yasu.   If you just want to eat salmon sashimi till you full, there are tons of ayce places you can go to on Bloor St.  I am just happy that Ouchi San decided to open Yasu. I have been to his previous restaurant Nigiri-Ya at Leaside it was good but not as good as what he is doing at Yasu now. $80 seems to be steep for sushi but trust me the experience more than worth it!   Fresh fish and seafood from all over the world including Japan, Europe, North America. Each fish is featured as a piece of art in nigiri sushi with just the perfect preparation method and condiments (fresh wasabi, soya sauce, yuzu salt, shiso, ginger....)   Procedure:  - watch the preparation - listen to the explanation - admire my food - smell my food which will further enhance my experience - use my hand and put it my mouth and eat slowly and just let my taste buds dance - making happy noises and smile - Repeat  I was sad that I miss the fresh  Hokkaido Uni but seriously Aburi O-toro, Ankimo, Hokkaido Scallop even Mackerel tasted so good...  I will be back.. I want to be back...   When am I coming back? (let me go check their availability on Yelp reservation!)"
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import re
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

remove_list = ['NN', 'IN', 'DT', 'MD', 'TO', 'PR', 'CC', 'WD', 'WP']

review1 = review1.lower()
word_tokens = word_tokenize(review1)

result = []
words = []

tag_l = pos_tag(word_tokens)

for w in tag_l:
  if w[1][:2] in remove_list :
    continue
  else:
    words.append(w[0])

stop_words = set(stopwords.words('english'))
stop_words.update(['\'re', '\'d', '\'t', '\'ll', '\'ve', '\'s','\'m', '!', '.', ',', '/', '?', '\"', '@', '%', '&', '*', '=','(',')','{','}', '-', '--', 'u', '...','$', '#', '*', '@', ':', ';', '[',']','~']) 
stop_words.remove('not')

for w in words:
  if w == 'n\'t':
    w = 'not'
  if w not in stop_words:
    a = re.sub('[^a-zA-Z ]', '', w)
    if a != '':
      result.append(a) #result에 불용어 제거된 tokens 존재

newresult=[]
for token in result:
  shortword = re.compile(r'\W*\b\w{1,2}\b')
  tk = shortword.sub('', token)
  if tk != '':
    newresult.append(tk)

tmpresult=[]
for review in newresult:
  if review[:3] != 'www':
    tmpresult.append(review)
newresult = tmpresult

# 길이 14 이상
notremove = ['unpretentious', 'unfortunately', 'approximately', 'complimentary', 'uncomfortable','disappointing']
lenreview = []
for review in newresult:
  if len(review) >= 13 and review not in notremove :
    continue
  else:
    lenreview.append(review)

newresult = lenreview

data1 = newresult
    
tmp_test=[]

if len(data1) > 49:
  tmp_test.append(data1[0:49])
else:
  tmp_test.append(data1)
X_test = tmp_test

print(len(X_test[0]))
print(X_test)
tokenizer_obj_test = Tokenizer()
tokenizer_obj_test.fit_on_texts(X_test)
sequences_test = tokenizer_obj_test.texts_to_sequences(X_test)
review_pad_test = pad_sequences(sequences_test, maxlen=49, padding = 'post')
#print(review_pad_test)

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
49
[['perfect', 'chinese', 'even', 'decades', 'far', 'perfect', 'survive', 'get', 'chinese', 'sure', 'not', 'say', 'rather', 'going', 'bed', 'empty', 'dirt', 'cheap', 'great', 'work', 'not', 'expect', 'much', 'cheap', 'chinese', 'live', 'throwing', 'stand', 'sliding', 'everywhere', 'understand', 'not', 'good', 'charging', 'not', 'stand', 'lukewarm', 'dim', 'shrimp', 'not', 'cold', 'like', 'hard', 'pan', 'fried', 'cake', 'oily', 'chicken', 'mai']]


In [3]:
from keras.models import load_model
# 2. 모델 불러오기1
model = load_model('/content/drive/My Drive/final/model1_85.29.h5')

# 3. 모델 사용하기
yhat = model.predict_classes(review_pad_test)







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [4]:
print(review1)

pre = yhat
if pre == 0:
  print("=> bad review")
else:
  print("=> good review")

perfect chinese restaurant has been around for ages, even decades! it is far from perfect but as a 24 hour business it does survive because where can you get some chinese food in the middle of the night in scarborough? but... for sure perfect will not be the place for me... can i say i rather going to bed on my empty stomach?   prices here for dim sum is dirt cheap $1.50 for all sizes all day weekday and weekends are $1.80. the specials are from 7 am, great if you have to work early.. i don't expect much services in cheap chinese restaurant. i can live with servers throwing things on our tables. i can stand the plastic table cover where all my plates are sliding everywhere. i can understand that the quality of food will not be as good with the price they are charging! what i cannot stand is lukewarm to room temperature dim sum!   shrimp rice roll - not only cold, the shrimps inside was like hard.  pan fried turnip cake - cold and oily.  chicken feet - lukewarm  siu mai - lukewarm  har 